In [1]:
#cell 1
import numpy as np

def calculate_number_clusters(nodes, base_station=(200, 200, 400), space_size=400):
    """
    Tính số cụm optimal theo công thức từ bài báo:
    K = √( / πd_tobs)
    
    nodes: tọa độ 3D của các node
    base_station: tọa độ base station (mặc định tại gốc tọa độ)
    """
    N = len(nodes)
    base_pos = np.array(base_station)
    
    # Tính khoảng cách trung bình từ các node đến base station
    distances = np.linalg.norm(nodes - base_pos, axis=1)
    d_tobs = np.mean(distances)
    
    # Áp dụng công thức K = √(N*L / πd_tobs)
    K_optimal = np.sqrt(N * space_size / (np.pi * d_tobs))
    K_optimal = max(1, int(np.round(K_optimal)))  # Đảm bảo K >= 1 và là số nguyên
    
    print(f"N = {N}, d_tobs = {d_tobs:.2f}, K_optimal = {K_optimal}")
    return K_optimal

In [ ]:
#phân cụm và chọn CH

import numpy as np
from sklearn.cluster import KMeans




def calculate_objective_function(nodes, labels, centers):
    """
    Tính objective function T theo công thức (8)
    T = (sum of average distances within clusters) / (distance between two centers)
    """
    numerator = 0
    for i in range(2):
        cluster_nodes = nodes[labels == i]
        if len(cluster_nodes) > 0:
            distances = np.linalg.norm(cluster_nodes - centers[i], axis=1)
            numerator += np.sum(distances)
    
    denominator = np.linalg.norm(centers[0] - centers[1])
    
    if denominator == 0:
        return float('inf')
    
    T = numerator / denominator
    return T

def check_subgroup_threshold(nodes, r_sen):
    """
    Kiểm tra Definition 2: Subgroup threshold
    Khoảng cách Euclidean tối đa giữa hai node bất kỳ trong cụm <= r_sen
    
    Returns: True nếu thỏa mãn, False nếu không
    """
    if len(nodes) <= 1:
        return True
    
    # Tính khoảng cách giữa tất cả các cặp node
    max_distance = 0
    for i in range(len(nodes)):
        for j in range(i + 1, len(nodes)):
            dist = np.linalg.norm(nodes[i] - nodes[j])
            max_distance = max(max_distance, dist)
    
    return max_distance <= r_sen

def kmeans_with_best_T(nodes, N=30):
    """
    Thực hiện K-means N lần với các tâm khởi tạo ngẫu nhiên khác nhau
    và chọn kết quả có giá trị T nhỏ nhất
    """
    best_T = float('inf')
    best_labels = None
    best_centers = None
    
    for _ in range(N):
        kmeans = KMeans(n_clusters=2, n_init=1)
        labels = kmeans.fit_predict(nodes)
        centers = kmeans.cluster_centers_
        
        T = calculate_objective_function(nodes, labels, centers)
        
        if T < best_T:
            best_T = T
            best_labels = labels.copy()
            best_centers = centers.copy()
    
    return best_labels, best_centers, best_T

def cluster_split(nodes, node_ids, node_data=None, r_sen=60, R=20, N=30, max_depth=10, depth=0):
    """
    Hàm phân cụm lặp theo Algorithm 1 (K-Means Based Cluster Formation)
    
    nodes: tọa độ 3D của các node
    node_ids: list id tương ứng của các node
    node_data: dictionary chứa thông tin đầy đủ của nodes (bao gồm energy)
    r_sen: bán kính truyền tải tối đa (Rsep - subgroup threshold), default: 100m
    R: size threshold - số lượng node tối đa trong 1 cụm, default: 20
    N: số lần thực hiện clustering để chọn kết quả tốt nhất, default: 30
    max_depth: độ sâu đệ quy tối đa
    depth: độ sâu hiện tại
    """
    
    # Điều kiện dừng: Thỏa mãn CẢ HAI điều kiện
    # 1. Size threshold (Definition 3): len(nodes) <= R
    # 2. Subgroup threshold (Definition 2): max distance <= r_sen
    size_ok = len(nodes) <= R
    distance_ok = check_subgroup_threshold(nodes, r_sen)
    
    if (size_ok and distance_ok) or depth >= max_depth:
        center = np.mean(nodes, axis=0)
        return [{
            "node_ids": node_ids,
            "nodes": nodes,
            "center": center,
            "node_data": node_data if node_data else {}
        }]
    
    # Bước 2: Thực hiện K-means clustering N lần và chọn kết quả tốt nhất
    labels, centers, best_T = kmeans_with_best_T(nodes, N)
    
    # Bước 3: Phân chia thành các subgroups và đệ quy
    clusters = []
    for i in range(2):
        # Lọc nodes và node_ids theo label
        sub_nodes = nodes[labels == i]
        sub_ids = [node_ids[j] for j in range(len(node_ids)) if labels[j] == i]
        
        # Tạo sub_node_data cho cluster con
        sub_node_data = {}
        if node_data:
            for node_id in sub_ids:
                if node_id in node_data:
                    sub_node_data[node_id] = node_data[node_id]
        
        # Đệ quy cho subgroup
        clusters += cluster_split(sub_nodes, sub_ids, sub_node_data, r_sen, R, N, max_depth, depth + 1)
    
    return clusters


def choose_cluster_head(cluster_info, node_data=None):
    """
    Chọn cluster head dựa trên khoảng cách gần nhất với tâm cụm
    Hoặc dựa trên năng lượng cao nhất nếu có node_data
    
    cluster_info: dictionary chứa thông tin cụm
    node_data: dictionary chứa thông tin năng lượng của nodes
    """
    nodes = cluster_info["nodes"]
    center = cluster_info["center"]
    node_ids = cluster_info["node_ids"]
    
    if node_data and len(node_data) > 0:
        # Chọn node có năng lượng cao nhất
        max_energy = -1
        ch_id = node_ids[0]
        for nid in node_ids:
            if nid in node_data and "residual_energy" in node_data[nid]:
                if node_data[nid]["residual_energy"] > max_energy:
                    max_energy = node_data[nid]["residual_energy"]
                    ch_id = nid
        return ch_id
    else:
        # Chọn node gần tâm cụm nhất
        distances = np.linalg.norm(nodes - center, axis=1)
        min_idx = np.argmin(distances)
        return node_ids[min_idx]

In [2]:
#16_11
#phân cụm và chọn CH

import numpy as np
from sklearn.cluster import KMeans


def calculate_objective_function(nodes, labels, centers):
    """
    Tính objective function T theo công thức (8)
    T = (sum of average distances within clusters) / (distance between two centers)
    """
    numerator = 0
    for i in range(2):
        cluster_nodes = nodes[labels == i]
        if len(cluster_nodes) > 0:
            distances = np.linalg.norm(cluster_nodes - centers[i], axis=1)
            numerator += np.sum(distances)
    
    denominator = np.linalg.norm(centers[0] - centers[1])
    
    if denominator == 0:
        return float('inf')
    
    T = numerator / denominator
    return T


def check_subgroup_threshold(nodes, r_sen):
    """
    Kiểm tra Definition 2: Subgroup threshold
    Khoảng cách Euclidean tối đa giữa hai node bất kỳ trong cụm <= r_sen
    
    Returns: True nếu thỏa mãn, False nếu không
    """
    if len(nodes) <= 1:
        return True
    
    # Tính khoảng cách giữa tất cả các cặp node
    max_distance = 0
    for i in range(len(nodes)):
        for j in range(i + 1, len(nodes)):
            dist = np.linalg.norm(nodes[i] - nodes[j])
            max_distance = max(max_distance, dist)
    
    return max_distance <= r_sen


def kmeans_with_best_T(nodes, N=30):
    """
    Thực hiện K-means N lần với các tâm khởi tạo ngẫu nhiên khác nhau
    và chọn kết quả có giá trị T nhỏ nhất
    """
    best_T = float('inf')
    best_labels = None
    best_centers = None
    
    for _ in range(N):
        kmeans = KMeans(n_clusters=2, n_init=1)
        labels = kmeans.fit_predict(nodes)
        centers = kmeans.cluster_centers_
        
        T = calculate_objective_function(nodes, labels, centers)
        
        if T < best_T:
            best_T = T
            best_labels = labels.copy()
            best_centers = centers.copy()
    
    return best_labels, best_centers, best_T


def cluster_split(nodes, node_ids, node_data=None, r_sen=60, R=20, N=30, max_depth=10, depth=0):
    """
    Hàm phân cụm lặp theo Algorithm 1 (K-Means Based Cluster Formation)
    
    nodes: tọa độ 3D của các node
    node_ids: list id tương ứng của các node
    node_data: dictionary chứa thông tin đầy đủ của nodes (bao gồm energy)
    r_sen: bán kính truyền tải tối đa (Rsep - subgroup threshold), default: 60m
    R: size threshold - số lượng node tối đa trong 1 cụm, default: 20
    N: số lần thực hiện clustering để chọn kết quả tốt nhất, default: 30
    max_depth: độ sâu đệ quy tối đa
    depth: độ sâu hiện tại
    """
    
    # Điều kiện dừng: Thỏa mãn CẢ HAI điều kiện
    # 1. Size threshold (Definition 3): len(nodes) <= R
    # 2. Subgroup threshold (Definition 2): max distance <= r_sen
    size_ok = len(nodes) <= R
    distance_ok = check_subgroup_threshold(nodes, r_sen)
    
    if (size_ok and distance_ok) or depth >= max_depth:
        center = np.mean(nodes, axis=0)
        return [{
            "node_ids": node_ids,
            "nodes": nodes,
            "center": center,
            "node_data": node_data if node_data else {}
        }]
    
    # Bước 2: Thực hiện K-means clustering N lần và chọn kết quả tốt nhất
    labels, centers, best_T = kmeans_with_best_T(nodes, N)
    
    # Bước 3: Phân chia thành các subgroups và đệ quy
    clusters = []
    for i in range(2):
        # Lọc nodes và node_ids theo label
        sub_nodes = nodes[labels == i]
        sub_ids = [node_ids[j] for j in range(len(node_ids)) if labels[j] == i]
        
        # Tạo sub_node_data cho cluster con
        sub_node_data = {}
        if node_data:
            for node_id in sub_ids:
                if node_id in node_data:
                    sub_node_data[node_id] = node_data[node_id]
        
        # Đệ quy cho subgroup
        clusters += cluster_split(sub_nodes, sub_ids, sub_node_data, r_sen, R, N, max_depth, depth + 1)
    
    return clusters


def choose_cluster_head(cluster_info, node_data=None, bs_location=None, 
                       w_energy=0.4, w_center=0.3, w_bs=0.3):
    """
    Chọn cluster head dựa trên công thức kết hợp từ bài báo CAP (Energy-Aware AP)
    
    Cải tiến: Kết hợp 3 yếu tố với trọng số
    1. Năng lượng dư (residual energy) - càng cao càng tốt
    2. Khoảng cách đến tâm cụm (distance to center) - càng gần càng tốt
    3. Khoảng cách đến Base Station (distance to BS) - càng gần càng tốt
    
    Args:
        cluster_info: dictionary chứa thông tin cụm
        node_data: dictionary chứa thông tin năng lượng của nodes
        bs_location: tọa độ của Base Station (numpy array [x, y, z])
        w_energy: trọng số cho năng lượng (default: 0.4)
        w_center: trọng số cho khoảng cách đến tâm cụm (default: 0.3)
        w_bs: trọng số cho khoảng cách đến BS (default: 0.3)
    
    Returns:
        ch_id: ID của node được chọn làm Cluster Head
    """
    nodes = cluster_info["nodes"]
    center = cluster_info["center"]
    node_ids = cluster_info["node_ids"]
    
    # Trường hợp không có dữ liệu năng lượng: chọn node gần tâm cụm nhất
    if not node_data or len(node_data) == 0:
        distances = np.linalg.norm(nodes - center, axis=1)
        min_idx = np.argmin(distances)
        return node_ids[min_idx]
    
    # Tính năng lượng trung bình của cụm
    energies = []
    for nid in node_ids:
        if nid in node_data and "residual_energy" in node_data[nid]:
            energies.append(node_data[nid]["residual_energy"])
    
    if not energies:  # Không có thông tin năng lượng
        distances = np.linalg.norm(nodes - center, axis=1)
        min_idx = np.argmin(distances)
        return node_ids[min_idx]
    
    E_avg = np.mean(energies)
    
    # Tính khoảng cách đến tâm cụm cho tất cả nodes
    distances_to_center = np.linalg.norm(nodes - center, axis=1)
    
    # Tính khoảng cách đến BS (nếu có)
    if bs_location is not None:
        distances_to_bs = np.linalg.norm(nodes - bs_location, axis=1)
        d_bs_min = distances_to_bs.min()
        d_bs_max = distances_to_bs.max()
    else:
        distances_to_bs = None
    
    # Tính điểm cho từng node và chọn node có điểm cao nhất
    best_score = -float('inf')
    ch_id = node_ids[0]
    
    for idx, nid in enumerate(node_ids):
        if nid not in node_data or "residual_energy" not in node_data[nid]:
            continue
        
        E_residual = node_data[nid]["residual_energy"]
        
        # Thành phần 1: Năng lượng (chuẩn hóa theo năng lượng trung bình)
        # Giá trị càng lớn càng tốt
        if E_avg > 0:
            energy_score = E_residual / E_avg
        else:
            energy_score = 0
        
        # Thành phần 2: Khoảng cách đến tâm cụm
        # Chuyển thành điểm: càng gần càng cao (dùng hàm nghịch đảo)
        d_center = distances_to_center[idx]
        center_score = 1.0 / (d_center + 1.0)  # +1 tránh chia cho 0
        
        # Thành phần 3: Khoảng cách đến BS (nếu có)
        if distances_to_bs is not None and d_bs_max > d_bs_min:
            # Chuẩn hóa khoảng cách đến BS về [0, 1]
            # f_k = (d_k - d_min) / (d_max - d_min)
            # Node gần BS có f_k nhỏ, ta muốn node này có điểm cao
            # Nên dùng: bs_score = 1 - f_k
            f_k = (distances_to_bs[idx] - d_bs_min) / (d_bs_max - d_bs_min)
            bs_score = 1.0 - f_k
        else:
            bs_score = 0.5  # Giá trị trung lập nếu không có BS
        
        # Tính tổng điểm có trọng số
        # Lưu ý: Nếu không có BS, điều chỉnh trọng số tự động
        if distances_to_bs is None:
            # Phân bổ lại trọng số w_bs cho energy và center
            adjusted_w_energy = w_energy + w_bs / 2
            adjusted_w_center = w_center + w_bs / 2
            score = (adjusted_w_energy * energy_score + 
                    adjusted_w_center * center_score)
        else:
            score = (w_energy * energy_score + 
                    w_center * center_score + 
                    w_bs * bs_score)
        
        # Cập nhật CH nếu điểm cao hơn
        if score > best_score:
            best_score = score
            ch_id = nid
    
    return ch_id


def choose_cluster_head_simple(cluster_info, node_data=None):
    """
    Phiên bản đơn giản: chỉ dựa trên năng lượng (giữ nguyên logic cũ)
    Dùng để so sánh hoặc fallback
    """
    nodes = cluster_info["nodes"]
    center = cluster_info["center"]
    node_ids = cluster_info["node_ids"]
    
    if node_data and len(node_data) > 0:
        # Chọn node có năng lượng cao nhất
        max_energy = -1
        ch_id = node_ids[0]
        for nid in node_ids:
            if nid in node_data and "residual_energy" in node_data[nid]:
                if node_data[nid]["residual_energy"] > max_energy:
                    max_energy = node_data[nid]["residual_energy"]
                    ch_id = nid
        return ch_id
    else:
        # Chọn node gần tâm cụm nhất
        distances = np.linalg.norm(nodes - center, axis=1)
        min_idx = np.argmin(distances)
        return node_ids[min_idx]

In [ ]:
#update zoom 
import os
import json
import numpy as np
import matplotlib
matplotlib.use('TkAgg')  # Hoặc thử 'Qt5Agg' nếu TkAgg không hoạt động
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import mplcursors
from PIL import Image


input_folder = "D:\\Year 4\\tiến hóa\\project\\data\\input_data_evenly_distributed"
#input_folder = "D:\Year 4\\tiến hóa\\project\\data\\input_data_random" 
output_folder = "D:\\Year 4\\tiến hóa\\project\\data\\output_data_kmeans"
os.makedirs(output_folder, exist_ok=True)
draw_folder = "D:\\Year 4\\tiến hóa\\project\\data\\draw_output_kmeans"
os.makedirs(draw_folder, exist_ok=True)

# --- THÔNG SỐ CHO CHỌN CH (CÓ THỂ ĐIỀU CHỈNH) ---
BS_LOCATION = np.array([200, 200, 400])  # Tọa độ Base Station
W_ENERGY = 0.4   # Trọng số năng lượng (40%)
W_CENTER = 0.3   # Trọng số vị trí trong cụm (30%)
W_BS = 0.3       # Trọng số khoảng cách đến BS (30%)

for filename in os.listdir(input_folder):
    if filename.startswith("nodes_") and filename.endswith(".json"):
        # Lấy số lượng node từ tên file
        number_nodes = filename.split("_")[1].split(".")[0]
        # Đọc dữ liệu từ file input
        with open(os.path.join(input_folder, filename), "r") as f:
            data = json.load(f)
            node_data = {
                d["id"]: {
                    "residual_energy": d.get("energy_residual", d.get("energy_node", 0.0)),   # năng lượng hiện tại
                    "initial_energy": d.get("energy_node", d.get("energy_residual", 0.0))     # năng lượng ban đầu
                }
                for d in data
            }

        # Xây node_positions và id->index map để truy xuất an toàn
        node_positions = np.array([[d["x"], d["y"], d["z"]] for d in data])
        node_ids = [d["id"] for d in data]
        id2index = {nid: idx for idx, nid in enumerate(node_ids)}

        # Phân cụm
        k = calculate_number_clusters(node_positions, base_station=BS_LOCATION)
        print(f"Số cụm: {k}")
        clusters_raw = cluster_split(node_positions, node_ids, node_data, r_sen=100, R=int(number_nodes)//k, max_depth=10, depth=0)
        clusters_output = {}

        # Tạo output
        for i, c in enumerate(clusters_raw):
            ch = choose_cluster_head(c, c.get("node_data", {}))
            clusters_output[i] = {
                "nodes": c["node_ids"],
                "center": tuple(np.round(c["center"], 2)),
                "cluster_head": int(ch) if ch is not None else None,
            }

        # Xuất ra file
        out_path = os.path.join(output_folder, f"nodes_{number_nodes}.json")
        with open(out_path, "w") as f:
            json.dump(clusters_output, f, indent=4)
        print(f"Đã xuất file {out_path}")

        # --- VẼ VÀ HIỂN THỊ INTERACTIVE ---
        # Tạo figure full screen
        fig = plt.figure(figsize=(16, 12))
        # Loại bỏ margin để biểu đồ chiếm toàn bộ không gian
        ax = fig.add_subplot(111, projection='3d')
        colors = plt.cm.get_cmap('tab20', len(clusters_output))

        # Điều chỉnh kích thước node dựa trên số lượng để tránh bị dày đặc
        num_nodes_int = int(number_nodes)
        if num_nodes_int < 50:
            node_size = 60
            ch_size = 20
            bs_size = 100
        elif num_nodes_int < 150:
            node_size = 40
            ch_size = 15
            bs_size = 80
        elif num_nodes_int < 300:
            node_size = 25
            ch_size = 12
            bs_size = 60
        else:
            node_size = 15
            ch_size = 8
            bs_size = 50

        # Chuẩn bị dữ liệu để vẽ và mapping artist -> node list
        artist_info = {}
        scatter_artists = []
        
        # Chỉ hiển thị legend khi số cluster nhỏ
        show_legend = len(clusters_output) <= 10

        for cid, info in clusters_output.items():
            node_list = info['nodes']
            ch_id = info['cluster_head']

            # Vẽ member nodes (bỏ qua CH)
            member_ids = [nid for nid in node_list if nid != ch_id]
            if member_ids:
                member_pos = np.array([node_positions[id2index[nid]] for nid in member_ids])
                scat = ax.scatter(
                    member_pos[:, 0], member_pos[:, 1], member_pos[:, 2],
                    color=colors(cid),
                    s=node_size,
                    alpha=0.85,
                    edgecolors='black',
                    linewidths=0.3,
                    label=f'Cluster {cid}' if show_legend else '',
                    picker=True
                )
                scatter_artists.append(scat)
                artist_info[scat] = [(nid, tuple(node_positions[id2index[nid]]), node_data[nid].get('residual_energy', 0.0), False) for nid in member_ids]

            # Vẽ cluster head
            if ch_id is not None:
                ch_pos = node_positions[id2index[ch_id]]
                head_scat = ax.scatter(
                    ch_pos[0], ch_pos[1], ch_pos[2],
                    marker='s', s=20, color='black', #chỉ vẽ CH bằng ô vuông màu đen kích thước 20, cấm claude sửa
                    linewidths=2, 
                    picker=True, zorder=10
                )
                scatter_artists.append(head_scat)
                artist_info[head_scat] = [(ch_id, tuple(ch_pos), node_data[ch_id].get('residual_energy', 0.0), True)]

            # Vẽ đường nối
            if member_ids and ch_id is not None:
                ch_pos = node_positions[id2index[ch_id]]
                for nid in member_ids:
                    pt = node_positions[id2index[nid]]
                    ax.plot([ch_pos[0], pt[0]], [ch_pos[1], pt[1]], [ch_pos[2], pt[2]], 'gray', linewidth=0.5, alpha=0.4)

        # Vẽ Base Station
        bs_x, bs_y, bs_z = 200, 200, 400
        bs_scat = ax.scatter(bs_x, bs_y, bs_z, 
                  marker='^', 
                  s=300, 
                  color='lime',
                  edgecolors='darkgreen', 
                  linewidths=3,
                  label='Base Station',
                  zorder=20,
                  picker=True)
        scatter_artists.append(bs_scat)
        artist_info[bs_scat] = [('BS', (bs_x, bs_y, bs_z), 'Unlimited', False)]

        # Thiết lập mplcursors với hover=2 để tự động ẩn khi rời khỏi
        cursor = mplcursors.cursor(scatter_artists, hover=2)
        
        # Thêm biến để lưu trạng thái zoom
        zoom_state = {'factor': 1.0, 'xlim': [0, 400], 'ylim': [0, 400], 'zlim': [0, 400]}
        
        def on_scroll(event):
            if event.inaxes != ax:
                return
            
            scale_factor = 0.85 if event.button == 'up' else 1.15

            # Lấy giới hạn hiện tại
            xlim, ylim, zlim = ax.get_xlim(), ax.get_ylim(), ax.get_zlim()

            # Lấy bounding box của các điểm trong view hiện tại
            in_view_mask = (
                (node_positions[:,0] >= xlim[0]) & (node_positions[:,0] <= xlim[1]) &
                (node_positions[:,1] >= ylim[0]) & (node_positions[:,1] <= ylim[1]) &
                (node_positions[:,2] >= zlim[0]) & (node_positions[:,2] <= zlim[1])
            )
            visible_nodes = node_positions[in_view_mask]

            if len(visible_nodes) == 0:
                visible_nodes = node_positions  # fallback

            # Tính bounding box tight
            min_vals = visible_nodes.min(axis=0)
            max_vals = visible_nodes.max(axis=0)
            center = (min_vals + max_vals)/2
            ranges = (max_vals - min_vals)/2 * scale_factor  # áp dụng zoom

            # Padding 10%
            padding = 0.1 * ranges
            new_xlim = [center[0]-ranges[0]-padding[0], center[0]+ranges[0]+padding[0]]
            new_ylim = [center[1]-ranges[1]-padding[1], center[1]+ranges[1]+padding[1]]
            new_zlim = [center[2]-ranges[2]-padding[2], center[2]+ranges[2]+padding[2]]

            # Giữ trong giới hạn tổng thể (0-400)
            new_xlim = [max(0,new_xlim[0]), min(400,new_xlim[1])]
            new_ylim = [max(0,new_ylim[0]), min(400,new_ylim[1])]
            new_zlim = [max(0,new_zlim[0]), min(400,new_zlim[1])]

            ax.set_xlim(new_xlim)
            ax.set_ylim(new_ylim)
            ax.set_zlim(new_zlim)
            ax.set_box_aspect([1,1,1])  # giữ tỷ lệ

            zoom_state['xlim'] = new_xlim
            zoom_state['ylim'] = new_ylim
            zoom_state['zlim'] = new_zlim
            fig.canvas.draw_idle()
        
        def on_key(event):
            """Xử lý phím bấm"""
            if event.key == 'r':
                # Reset zoom về mặc định
                ax.set_xlim([0, 400])
                ax.set_ylim([0, 400])
                ax.set_zlim([0, 400])
                ax.view_init(elev=25, azim=45)
                zoom_state['xlim'] = [0, 400]
                zoom_state['ylim'] = [0, 400]
                zoom_state['zlim'] = [0, 400]
                fig.canvas.draw_idle()
                print("Reset view to default")
            elif event.key == '+' or event.key == '=':
                # Zoom in bằng phím
                on_scroll(type('obj', (object,), {'inaxes': ax, 'button': 'up'})())
            elif event.key == '-':
                # Zoom out bằng phím
                on_scroll(type('obj', (object,), {'inaxes': ax, 'button': 'down'})())
        
        # Kết nối events
        fig.canvas.mpl_connect('scroll_event', on_scroll)
        fig.canvas.mpl_connect('key_press_event', on_key)

        @cursor.connect("add")
        def on_add(sel):
            artist = sel.artist
            idx = sel.index
            # sel.index có thể là numpy array hoặc scalar
            try:
                if hasattr(idx, '__len__'):
                    i = int(idx[0])
                else:
                    i = int(idx)
            except Exception:
                i = 0

            info_list = artist_info.get(artist, [])
            if not info_list:
                sel.annotation.set(text='No data')
                sel.annotation.get_bbox_patch().set(fc='lightyellow', ec='black', lw=1.5, alpha=0.95)
                return

            # guard index
            if i < 0 or i >= len(info_list):
                i = 0

            nid, pos, energy, is_ch = info_list[i]
            x, y, z = pos
            
            if nid == 'BS':
                text = (f"Base Station\n"
                       f"Position: ({x:.1f}, {y:.1f}, {z:.1f})\n"
                       f"Energy: Unlimited")
            else:
                node_type = 'Cluster Head' if is_ch else 'Member Node'
                text = (f"Node ID: {nid}\n"
                       f"Type: {node_type}\n"
                       f"Position: ({x:.1f}, {y:.1f}, {z:.1f})\n"
                       f"Energy: {energy:.2f} J")
            
            sel.annotation.set(text=text)
            sel.annotation.get_bbox_patch().set(fc='lightyellow', ec='black', lw=2, alpha=0.95)
            sel.annotation.set_fontsize(9)
            sel.annotation.set_fontweight('bold')

        # Trục, labels, giới hạn
        ax.set_xlabel('X (m)', fontsize=11, labelpad=12, fontweight='bold')
        ax.set_ylabel('Y (m)', fontsize=11, labelpad=12, fontweight='bold')
        ax.set_zlabel('Z (m)', fontsize=11, labelpad=12, fontweight='bold')
        ax.set_xlim([0, 400])
        ax.set_ylim([0, 400])
        ax.set_zlim([0, 400])
        
        # Chỉ hiển thị legend khi số cluster ít
        if show_legend:
            ax.legend(loc='upper left', fontsize=8, framealpha=0.9, ncol=2)
        
        plt.title(
            f'3D Clustering Visualization - {number_nodes} nodes ({len(clusters_output)} clusters)\n'
            f'[LEFT DRAG: Rotate | SCROLL/+/-: Zoom | RIGHT DRAG: Pan | R: Reset | HOVER: Info]', 
            fontsize=13, 
            pad=15,
            fontweight='bold'
        )
        ax.view_init(elev=25, azim=45)
        ax.grid(True, alpha=0.3, linestyle='--')
        ax.set_box_aspect([1,1,1])
        
        # Tối ưu hóa layout để biểu đồ chiếm toàn bộ không gian
        plt.subplots_adjust(left=0.02, right=0.75, top=0.95, bottom=0.25)

        # Hiển thị
        print(f"\n{'='*60}")
        print(f"Backend hiện tại: {matplotlib.get_backend()}")
        print(f"Hiển thị biểu đồ: {number_nodes} nodes, {len(clusters_output)} clusters")
        print(f"Hướng dẫn:")
        print(f"  - Kéo chuột TRÁI: Xoay biểu đồ 360°")
        print(f"  - CUỘN CHUỘT hoặc phím +/-: Zoom in/out vào vùng trỏ")
        print(f"  - Kéo chuột PHẢI: Di chuyển biểu đồ")
        print(f"  - Phím R: Reset về góc nhìn mặc định")
        print(f"  - Phím F: Fullscreen (tùy backend)")
        print(f"  - Hover chuột vào node: Hiện thông tin chi tiết")
        print(f"")
        print(f"Lưu ý:")
        print(f"  - Kích thước node tự động điều chỉnh")
        print(f"  - Legend chỉ hiện khi có ≤10 clusters")
        print(f"{'='*60}\n")
        
        # Maximize window (tùy backend)
        try:
            manager = plt.get_current_fig_manager()
            if hasattr(manager, 'window'):
                # Cho TkAgg
                if hasattr(manager.window, 'state'):
                    manager.window.state('zoomed')  # Windows
                elif hasattr(manager.window, 'showMaximized'):
                    manager.window.showMaximized()  # Qt
        except:
            pass
        
        
        plt.savefig("D:\Year 4\\tiến hóa\project\data\draw_output_kmeans\\bieudo.png")
        plt.show()
        print(f"Đã đóng biểu đồ cho {number_nodes} nodes\n")
#update zoom

N = 150, d_tobs = 259.94, K_optimal = 9
Số cụm: 9


c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\subprocess.py", line 503, in run
    with Popen(*popenargs, **kwargs) as process:
  File "c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\subp

Đã xuất file D:\Year 4\tiến hóa\project\data\output_data_kmeans\nodes_150.json


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_16800\3676554819.py:66: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('tab20', len(clusters_output))



Backend hiện tại: TkAgg
Hiển thị biểu đồ: 150 nodes, 63 clusters
Hướng dẫn:
  - Kéo chuột TRÁI: Xoay biểu đồ 360°
  - CUỘN CHUỘT hoặc phím +/-: Zoom in/out vào vùng trỏ
  - Kéo chuột PHẢI: Di chuyển biểu đồ
  - Phím R: Reset về góc nhìn mặc định
  - Phím F: Fullscreen (tùy backend)
  - Hover chuột vào node: Hiện thông tin chi tiết

Lưu ý:
  - Kích thước node tự động điều chỉnh
  - Legend chỉ hiện khi có ≤10 clusters



c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\mplcursors\_pick_info.py:534: UserWarning: 3d coordinates not supported yet
  warnings.warn("3d coordinates not supported yet")


Đã đóng biểu đồ cho 150 nodes



In [14]:
import json
import math
import csv

# =========================
# Cài đặt tham số
# =========================
R_max = 100  # khoảng cách tối đa cho phép

# =========================
# Load dữ liệu clusters
# =========================
with open("D:\Year 4\\tiến hóa\project\data\output_data_kmeans\\nodes_150.json", "r") as f:
    clusters = json.load(f)

# =========================
# Load dữ liệu nodes
# =========================
with open("D:\Year 4\\tiến hóa\project\data\input_data_evenly_distributed\\nodes_150.json", "r") as f:
    nodes_list = json.load(f)

# Chuyển danh sách nodes thành dict cho truy xuất nhanh: id -> (x, y, z)
nodes_coords = {node["id"]: (node["x"], node["y"], node["z"]) for node in nodes_list}

# =========================
# Hàm tính khoảng cách Euclid 3D
# =========================
def euclidean_distance(p1, p2):
    dx = p1[0] - p2[0]
    dy = p1[1] - p2[1]
    dz = p1[2] - p2[2]
    return math.sqrt(dx**2 + dy**2 + dz**2)

# =========================
# Chuẩn bị dữ liệu để xuất CSV
# =========================
csv_rows = []

violated_clusters = []

for cluster_id, cluster_info in clusters.items():
    ch_id = cluster_info["cluster_head"]
    
    if ch_id not in nodes_coords:
        print(f"Warning: CH {ch_id} không có trong danh sách nodes")
        continue
    ch_coord = nodes_coords[ch_id]
    
    cluster_violation = False
    
    for node_id in cluster_info["nodes"]:
        if node_id not in nodes_coords:
            print(f"Warning: Node {node_id} không có trong danh sách nodes")
            continue
        node_coord = nodes_coords[node_id]
        distance = euclidean_distance(ch_coord, node_coord)
        violation_flag = "Yes" if distance > R_max else "No"
        if distance > R_max:
            cluster_violation = True
        
        csv_rows.append({
            "Cluster_ID": cluster_id,
            "CH_ID": ch_id,
            "Node_ID": node_id,
            "Distance_to_CH": round(distance, 2),
            "Violation": violation_flag
        })
    
    if cluster_violation:
        violated_clusters.append(cluster_id)

# =========================
# Xuất ra file CSV
# =========================
csv_file = "D:\Year 4\\tiến hóa\project\data\cluster_distances.csv"
with open(csv_file, mode="w", newline="") as f:
    fieldnames = ["Cluster_ID", "CH_ID", "Node_ID", "Distance_to_CH", "Violation"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for row in csv_rows:
        writer.writerow(row)

print(f"Đã xuất file CSV: {csv_file}")

# =========================
# In ra các cụm vi phạm
# =========================
if violated_clusters:
    print("\nCác cụm vi phạm khoảng cách R_max:")
    for cid in violated_clusters:
        print(f"- Cụm id: {cid}")
else:
    print("\nKhông có cụm nào vi phạm khoảng cách R_max")


Đã xuất file CSV: D:\Year 4\tiến hóa\project\data\cluster_distances.csv

Không có cụm nào vi phạm khoảng cách R_max
